Google Colab: https://colab.research.google.com/drive/1DOIOoSUj7CHB386dpCZqKWng7x3ls27v?usp=sharing

In [5]:
cd drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [6]:
ls

'Bank Customer Segmentation.ipynb'   first.ipynb    sqli.csv
'Copy of Assignment2.ipynb'          SQLBIG.ipynb   Untitled1.ipynb
'Customer Segmentation.ipynb'        sqli2.csv


In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import randint
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import classification_report
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [8]:
df=pd.read_csv('sqli.csv', encoding='utf-16')
df.head()

,Sentence,Label
0,NaN,1
1,""" or pg_sleep ( __TIME__ ) --",1
2,create user name identified by pass123 tempora...,1
3,%29,1
4,' AND 1 = utl_inaddr.get_host_address ( ( S...,1


In [9]:
df2=pd.read_csv("sqli2.csv")
df2.head()

,Query,Label
0,""" or pg_sleep ( __TIME__ ) --",1
1,create user name identified by pass123 tempora...,1
2,AND 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1


In [10]:
print(df.shape)
print(df2.shape)

(33761, 2)
(30919, 2)


In [11]:
df['Sentence'] = df['Sentence'].astype(str)
df['Sentence'] = df['Sentence'].str.lower()
df['Label'] = pd.to_numeric(df['Label'], errors='coerce')
df['Label'] = df['Label'].astype('Int64')

In [12]:
df2['Query'] = df2['Query'].astype(str)
df2['Query'] = df2['Query'].str.lower()
df2['Label'] = pd.to_numeric(df2['Label'], errors='coerce')
df2['Label'] = df2['Label'].astype('Int64')

In [13]:
df.rename(columns={'Sentence': 'Query'}, inplace=True)
df.head()

,Query,Label
0,nan,1
1,""" or pg_sleep ( __time__ ) --",1
2,create user name identified by pass123 tempora...,1
3,%29,1
4,' and 1 = utl_inaddr.get_host_address ( ( s...,1


In [14]:
df.drop_duplicates()
df.dropna()

,Query,Label
0,nan,1
1,""" or pg_sleep ( __time__ ) --",1
2,create user name identified by pass123 tempora...,1
3,%29,1
4,' and 1 = utl_inaddr.get_host_address ( ( s...,1
...,...,...
33756,syrett,0
33757,arrechea bellveh,0
33758,1664,0
33759,almaluez,0


In [15]:
df2.drop_duplicates()
df2.dropna()

,Query,Label
0,""" or pg_sleep ( __time__ ) --",1
1,create user name identified by pass123 tempora...,1
2,and 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1
...,...,...
30914,delete from door where grow = 'small',0
30915,delete from tomorrow,0
30916,select wide ( s ) from west,0
30917,select * from ( select slide from breath ),0


In [20]:
n = min(len(df), len(df2))

sample_0 = df.sample(n=n, random_state=42)
sample_1 = df2.sample(n=n, random_state=27)

Final_df_ = pd.concat([sample_0, sample_1], ignore_index=True)
print(Final_df_.shape)
Final_df_.drop_duplicates()
print(Final_df_.shape)


(61838, 2)
(61838, 2)


In [21]:
Final_df_.head()

,Query,Label
0,häußler commented flood found gechingen unpre...,0
1,sean,0
2,lamarr@tardos.mobi,0
3,"-7069 ) union all select 6638,6638,6638,6638,...",1
4,4381135407580429,0


In [22]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, vectorizer):
        self.texts = texts
        self.labels = labels
        self.vectorizer = vectorizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        text_vectorized = self.vectorizer.transform([text]).toarray()
        return torch.tensor(text_vectorized), torch.tensor(label)



texts = Final_df_['Query'].tolist()
labels = Final_df_['Label'].tolist()

texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

vectorizer = CountVectorizer()
vectorizer.fit(texts_train)

train_dataset = CustomDataset(texts_train, labels_train, vectorizer)
test_dataset = CustomDataset(texts_test, labels_test, vectorizer)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [23]:
test_loader

In [24]:
from transformers import BertTokenizer

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Split data into train and test sets
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Create custom datasets
train_dataset = CustomDataset(texts_train, labels_train, tokenizer)
test_dataset = CustomDataset(texts_test, labels_test, tokenizer)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [25]:
from transformers import DistilBertForSequenceClassification
from torch.optim import AdamW
from tqdm import tqdm
import gc

gc.collect()
torch.cuda.empty_cache()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

model.train()
num_epochs = 4

for epoch in range(num_epochs):
    epoch_loss = 0.0
    correct_preds = 0
    total_preds = 0

    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        _, predicted = torch.max(logits, dim=1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_accuracy = correct_preds / total_preds
    print(f"Epoch {epoch + 1}/{num_epochs} | Loss: {epoch_loss:.4f} | Accuracy: {epoch_accuracy:.4f}")


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/4: 100%|██████████| 12368/12368 [12:40<00:00, 16.26batch/s]


Epoch 1/4 | Loss: 202.3371 | Accuracy: 0.9960


...

Epoch 2/4 | Loss: 66.6380 | Accuracy: 0.9986


Epoch 3/4: 100%|██████████| 12368/12368 [12:45<00:00, 16.16batch/s]


Epoch 3/4 | Loss: 48.0337 | Accuracy: 0.9991


Epoch 4/4: 100%|██████████| 12368/12368 [12:45<00:00, 16.15batch/s]

Epoch 4/4 | Loss: 51.0166 | Accuracy: 0.9992


In [26]:
# Evaluation
model.eval()
eval_loss = 0.0
correct_preds = 0
total_preds = 0

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Evaluation', unit='batch'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        _, predicted = torch.max(logits, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

        eval_loss += loss.item()

eval_accuracy = correct_preds / total_preds
print(f'Evaluation Loss: {eval_loss}, Evaluation Accuracy: {eval_accuracy}')


Evaluation: 100%|██████████| 3092/3092 [00:56<00:00, 54.68batch/s]

Evaluation Loss: 35.91815807698049, Evaluation Accuracy: 0.9985446313065977


In [27]:
from transformers import DistilBertTokenizer

model.save_pretrained("sqlin_model")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer.save_pretrained("sqlin_model")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

('sqlin_model/tokenizer_config.json',
 'sqlin_model/special_tokens_map.json',
 'sqlin_model/vocab.txt',
 'sqlin_model/added_tokens.json')

In [28]:
import shutil
shutil.make_archive("sqlin_model", 'zip', "sqlin_model")

'/content/drive/MyDrive/Colab Notebooks/sqlin_model.zip'

In [29]:
from google.colab import files
files.download("sqlin_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>